# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [1]:
import numpy as np
def get_threshold(feature, y):
    best_thr = None
    best_e = 100000
    ind = np.argsort(feature)
    y = y[ind]
    feature = feature[ind]
    y_l = 0
    y2_l = 0
    n_l = 1
    y_r = sum(y)
    y2_r = y.dot(y)
    n_r = len(y)
    ind = 0
    for i in range(len(feature)):
        y_l = y_l + y[i]
        y2_l = y2_l + y[i]*y[i]
        n_l = n_l+1
        y_r = y_r - y[i]
        y2_r = y2_r - y[i]*y[i]
        n_r = n_r-1
        err = 0
        if (n_l!=0):
            err = (y2_l-y_l*y_l)/n_l
        if (n_r!=0):
            err = err + (y2_r-y_r*y_r)/n_r
        if (err<best_e):
            best_e = err
            ind = i
    best_thr = (np.std(y)*np.std(y) - best_e,feature[ind])
    return best_thr


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_q = -float('inf')
    for j in range(X.shape[1]):
        f_score, f_thr = get_threshold(X[:, j], y)
        if f_score > best_q:
            best_q = f_score
            best_f = j
            best_thr = f_thr
    return best_f, best_thr


    

In [2]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [4]:
%%time
j, thr = train_stump(X, y)
ind = X[:,j]>thr
y_pred = np.ones(len(y))
y_pred[ind] = np.mean(y[ind])
ind = X[:,j]<=thr
y_pred[ind] = np.mean(y[ind])
np.std(y-y_pred)**2

Wall time: 21 ms


46.34883428864869